In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import os
import cv2 
from google.colab.patches import cv2_imshow
from PIL import Image
import sys
import math
import random
from skimage.transform import rotate, AffineTransform
from skimage.util import random_noise
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import torch
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, ConcatDataset
import random
import tensorflow as tf
from tensorflow import keras
from torchvision import datasets

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Data Preparation (Augmentation and Normalization)

In [3]:
# this cell is regarding the aumentation and normalization with the stats of data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(degrees=45),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2)),
    transforms.RandomAffine(degrees=0, translate=(0.15, 0.15)),
    # transforms.RandomResizedCrop(size=(224, 224), scale=(0.5, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
dataset = datasets.ImageFolder("/content/drive/MyDrive/projectDataset/animal90/animals", transform= transform)

In [7]:
# this cell is spliting dataset to train and test
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
test_split = 0.2
dataset_size = len(dataset)
test_size = int(test_split * dataset_size)
train_size = dataset_size - test_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [8]:
# this cell is spliting train to train and val
val_split = 0.2
train_size = len(train_dataset)
val_size = int(val_split * train_size)
train_size = train_size - val_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

In [9]:
# Define the indices to use for the train and test samplers
train_indices = list(range(train_size))
val_indices = list(range(train_size, train_size+ val_size))
test_indices = list(range(train_size+ val_size, dataset_size))

# Define the samplers for the train and test sets
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

# Define the data loaders for the train and test sets
train_loader = DataLoader(train_dataset, batch_size=128, sampler=train_sampler,
                          drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=128, sampler=val_sampler,
                        drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=128, sampler=test_sampler,
                         drop_last=True)

In [10]:
for train, label in train_loader:
    print(train.shape)
    break

torch.Size([128, 3, 224, 224])
